In [18]:
invoice_template = """
# ${myLegalName}

---------------------------------------

    Invoice: ${invoiceNumber}
    Company: ${clientLegalName}
    Contact: ${clientLegalAddress}

    Developer: ${myLegalName}
    Contact: ${myclientLegalAddress}
    
### Summary {#summary}

Task Description                 | Amount | Cost  | Sub Total
-------------------------------- | ------ | --------- | ---------
${itemName}                      | ${itemCount}      | ${itemCost}    | ${itemTotal}


*Sub Total*: ${subTotal} / **Grand Total**: ${grandTotal} (no tax)

### Terms {#terms}

+ Payments are to be made payable to ${myLegalName} via Paypal[^1] or Direct Deposit[^2].
+ Grand Total must be paid by the end of ${today date+30 day} (30 days).
+ If Grand Total is not paid by the end of ${today date+30 day}, an late-fee[^3] will be applied to the Grand Total.

[^1]: Paypal e-mail address for payments is <[YOUR EMAIL]>.
[^2]: Please contact if you wish to do a Direct Deposit.
[^3]: Late-fee of *2%* interest per-day until paid.
"""

In [19]:
client_legal_entity = {
    "credentialSubject": {
        "id": "did:ebsi:123456789abcdefghi",
        "legalPersonIdentifier": "FR123456789",
        "legalName": "Acme Corporation",
        "legalAddress": "123 Acme Street, Paris, France",
        "VATRegistration": "FR123456789012",
        "taxReference": "FR1234567890",
        "LEI": "0123456789ABCDEFGHIJKLM",
        "EORI": "FR123456789012345",
        "SEED": "FR1234567890",
        "SIC": "0111",
        "domainName": "acme-corporation.com"
    }
}
my_legal_entity = {
    "credentialSubject": {
        "id": "did:ebsi:987654321abcdefghi",
        "legalPersonIdentifier": "DE987654321",
        "legalName": "Tech Innovators GmbH",
        "legalAddress": "456 Tech Street, Berlin, Germany",
        "VATRegistration": "DE987654321098",
        "taxReference": "DE9876543210",
        "LEI": "9876543210ABCDEFGHIJKLM",
        "EORI": "DE987654321098765",
        "SEED": "DE9876543210",
        "SIC": "0112",
        "domainName": "tech-innovators.com"
    }
}


In [31]:
def productDetails(name = "pipe"):
    return {
        "name": "pipe",
        "cost": 5,
        "currency": "EUR"
    }

def invoiceTemplate(name = "default"):
    return invoice_template

# https://ec.europa.eu/digital-building-blocks/wikis/display/EBSIDOC/Verifiable+ID+-+Legal+Entity
# Local Business
# https://schema.org/LocalBusiness

def myData(name= "Acme Corporation"):
    return my_legal_entity

def clientData(name = "Tech Innovators GmbH"):
    return client_legal_entity



In [38]:
from langchain.tools.render import render_text_description
from langchain.agents import initialize_agent, AgentType, Tool
from langchain.memory import ConversationBufferMemory

from langchain.tools.render import render_text_description
from langchain.agents.output_parsers import ReActSingleInputOutputParser
from langchain.agents.format_scratchpad import format_log_to_str
from langchain import hub

from langchain.agents import AgentExecutor

from langchain.chat_models import ChatOpenAI

tools = [
    Tool(
        name="Lookup for invoice template",
        func=invoiceTemplate,
        description="useful for when you need to get invoice template and replace all ${} with acutal data"
    ),
    Tool(
        name="Lookup for product",
        func=productDetails,
        description="useful for when you need to get product list"
    ),
    Tool(
        name="Lookup for client data",
        func=clientData,
        description="useful for when you need to get client data"
    ),
    Tool(
        name="Lookup for my data",
        func=myData,
        description="useful for when you need to get data about my company"
    ),
]

memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
llm = ChatOpenAI(temperature=0, model_name="gpt-4")

prompt = hub.pull("hwchase17/react-chat")
prompt = prompt.partial(
    tools=render_text_description(tools),
    tool_names=", ".join([t.name for t in tools]),
)

llm_with_stop = llm.bind(stop=["\nObservation"])
agent = {
    "input": lambda x: x["input"],
    "agent_scratchpad": lambda x: format_log_to_str(x['intermediate_steps']),
    "chat_history": lambda x: x["chat_history"]
} | prompt | llm_with_stop | ReActSingleInputOutputParser()

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True, memory=memory)
agent_executor.invoke({"input": "Generate Invoice from me for six pipe proucts for client named Acme Corporation"})['output']



> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? Yes
Action: Lookup for my data
Action Input: {}{'credentialSubject': {'id': 'did:ebsi:987654321abcdefghi', 'legalPersonIdentifier': 'DE987654321', 'legalName': 'Tech Innovators GmbH', 'legalAddress': '456 Tech Street, Berlin, Germany', 'VATRegistration': 'DE987654321098', 'taxReference': 'DE9876543210', 'LEI': '9876543210ABCDEFGHIJKLM', 'EORI': 'DE987654321098765', 'SEED': 'DE9876543210', 'SIC': '0112', 'domainName': 'tech-innovators.com'}}Do I need to use a tool? Yes
Action: Lookup for product
Action Input: {"product_name": "pipe"}{'name': 'pipe', 'cost': 5, 'currency': 'EUR'}Do I need to use a tool? Yes
Action: Lookup for client data
Action Input: {"client_name": "Antonio Salesmacher"}{'credentialSubject': {'id': 'did:ebsi:123456789abcdefghi', 'legalPersonIdentifier': 'FR123456789', 'legalName': 'Acme Corporation', 'legalAddress': '123 Acme Street, Paris, France', 'VATRegistration': 'FR123456789012', 'taxRefere

"Here is the invoice you requested:\n\n# Tech Innovators GmbH\n\n---------------------------------------\n\n    Invoice: [Invoice Number]\n    Company: Acme Corporation\n    Contact: 123 Acme Street, Paris, France\n\n    Developer: Tech Innovators GmbH\n    Contact: 456 Tech Street, Berlin, Germany\n    \n### Summary {#summary}\n\nTask Description                 | Amount | Cost  | Sub Total\n-------------------------------- | ------ | --------- | ---------\npipe                             | 6      | 5 EUR    | 30 EUR\n\n\n*Sub Total*: 30 EUR / **Grand Total**: 30 EUR (no tax)\n\n### Terms {#terms}\n\n+ Payments are to be made payable to Tech Innovators GmbH via Paypal[^1] or Direct Deposit[^2].\n+ Grand Total must be paid by the end of [Today's Date + 30 days] (30 days).\n+ If Grand Total is not paid by the end of [Today's Date + 30 days], a late-fee[^3] will be applied to the Grand Total.\n\n[^1]: Paypal e-mail address for payments is <[YOUR EMAIL]>.\n[^2]: Please contact if you wis